# 競馬予想アプリ - 当日予測 v2

**最終更新: 2026-01-25 19:15（個別レース予測に一本化）**

---

**Option B: 事前計算済み特徴量を使用**

## アーキテクチャ

```
【事前準備（ローカルPC - 週1回）】
1. モデル学習 → Supabase Storage にアップロード
2. 特徴量計算 → Supabase DB (horse_features) にアップロード

【当日予測（このノートブック - スマホからOK）】
1. Supabase Storage からモデルをダウンロード
2. netkeiba から当日のレース一覧を取得
3. レースを選択して個別予測を実行
4. JRA公式からリアルタイムオッズを取得（Playwright）
5. 予測結果を表示
```

## 使用方法
1. 左サイドバーの🔑からシークレットに以下を設定：
   - `SUPABASE_URL`
   - `SUPABASE_KEY`
2. セルを順番に実行（セクション1〜6）
3. セクション7でレース番号を指定して予測実行
4. オッズはレース直前に取得されるため、発走前に実行推奨

## 1. 環境セットアップ

In [ ]:
# 必要なパッケージをインストール
!pip install -q supabase requests beautifulsoup4 lxml pandas numpy lightgbm scikit-learn

In [ ]:
# Supabase認証情報の設定
try:
    from google.colab import userdata
    SUPABASE_URL = userdata.get('SUPABASE_URL') or "https://khpoovshkhppadsirgcp.supabase.co"
    SUPABASE_KEY = userdata.get('SUPABASE_KEY')
    if SUPABASE_KEY:
        print("✓ Colabシークレットから認証情報を取得しました")
    else:
        raise Exception("SUPABASE_KEY not set")
except:
    # 直接入力（テスト用）
    SUPABASE_URL = "https://khpoovshkhppadsirgcp.supabase.co"  # @param {type:"string"}
    SUPABASE_KEY = "your-service-role-key"  # @param {type:"string"}
    print("⚠ 手動入力の認証情報を使用します")

# 接続テスト
from supabase import create_client
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
print(f"✓ Supabase接続: {SUPABASE_URL[:40]}...")

## 2. モデルのダウンロード

In [ ]:
import pickle
import io
import pandas as pd
import numpy as np

BUCKET_NAME = "models"

def list_available_models():
    """利用可能なモデル一覧を取得"""
    try:
        files = supabase.storage.from_(BUCKET_NAME).list()
        models = [f["name"] for f in files if f["name"].endswith(".pkl")]
        print(f"利用可能なモデル: {models}")
        return models
    except Exception as e:
        print(f"エラー: {e}")
        return []

def download_model(version: str = "v1"):
    """モデルをダウンロード"""
    filename = f"model_{version}.pkl"
    print(f"モデルをダウンロード中: {filename}")

    try:
        response = supabase.storage.from_(BUCKET_NAME).download(filename)
        buffer = io.BytesIO(response)
        model_data = pickle.load(buffer)
        print(f"✓ モデルをダウンロードしました")
        print(f"  バージョン: {model_data.get('model_version', 'unknown')}")
        print(f"  特徴量数: {len(model_data.get('feature_columns', []))}")
        return model_data
    except Exception as e:
        print(f"✗ ダウンロード失敗: {e}")
        return None

# 利用可能なモデル一覧を表示
list_available_models()

In [ ]:
# モデルをダウンロード
MODEL_VERSION = "v20260113"  # @param {type:"string"}

model_data = download_model(MODEL_VERSION)

if model_data:
    model = model_data["model"]
    scaler = model_data["scaler"]
    calibrator = model_data.get("calibrator")
    feature_columns = model_data["feature_columns"]
    print(f"\n✓ モデル準備完了（{len(feature_columns)}特徴量）")
else:
    print("\n✗ モデルのダウンロードに失敗しました")
    print("ローカルFastAPIでモデルを学習・アップロードしてください")

## 3. 事前計算済み特徴量の確認

In [ ]:
# 特徴量テーブルの件数を確認
try:
    horse_result = supabase.table("horse_features").select("id", count="exact").execute()
    jockey_result = supabase.table("jockey_features").select("id", count="exact").execute()

    print(f"✓ horse_features: {horse_result.count}件")
    print(f"✓ jockey_features: {jockey_result.count}件")

    if horse_result.count == 0:
        print("\n⚠ 特徴量が同期されていません！")
        print("ローカルFastAPIで POST /api/v1/model/sync-features を実行してください")
except Exception as e:
    print(f"✗ テーブル確認エラー: {e}")
    print("Supabaseでマイグレーション（003_horse_features.sql）を実行してください")

## 4. 当日レースデータの取得

In [ ]:
import requests
import time
import re
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup
from typing import Optional, List, Dict

# ブラウザ自動化設定（Colab用）
USE_BROWSER = True  # @param {type:"boolean"} ブラウザでオッズを取得

if USE_BROWSER:
    print("ブラウザ環境をセットアップ中...")
    
    # google-colab-selenium: Colab専用パッケージ
    !pip install -q google-colab-selenium
    
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    import google_colab_selenium as gs
    
    def get_chrome_driver():
        """Colab用Chrome WebDriverを取得（google-colab-selenium使用）"""
        options = Options()
        options.add_argument("--headless=new")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-gpu")
        options.add_argument("--window-size=1920,1080")
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
        
        driver = gs.Chrome(options=options)
        return driver
    
    def close_browser(driver):
        """ブラウザを終了"""
        if driver:
            driver.quit()
    
    # 動作確認
    try:
        print("  ChromeDriverをセットアップ中...")
        test_driver = get_chrome_driver()
        test_driver.get("https://www.google.com")
        title = test_driver.title
        test_driver.quit()
        print(f"✓ ブラウザ準備完了（テストページ: {title}）")
    except Exception as e:
        print(f"⚠ ブラウザ初期化エラー: {e}")
        print("→ USE_BROWSER = False に設定してAPIフォールバックを使用します")
        USE_BROWSER = False

# スクレイピング設定
SCRAPE_INTERVAL = 1.5
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}

JRA_COURSE_CODES = {"01", "02", "03", "04", "05", "06", "07", "08", "09", "10"}
COURSE_NAMES = {
    "01": "札幌", "02": "函館", "03": "福島", "04": "新潟",
    "05": "東京", "06": "中山", "07": "中京", "08": "京都",
    "09": "阪神", "10": "小倉",
}

session = requests.Session()
session.headers.update(HEADERS)
last_request_time = 0

def fetch_html(url: str) -> str:
    global last_request_time
    elapsed = time.time() - last_request_time
    if elapsed < SCRAPE_INTERVAL:
        time.sleep(SCRAPE_INTERVAL - elapsed)

    response = session.get(url, timeout=30)
    last_request_time = time.time()

    if "EUC-JP" in response.text[:500] or "euc-jp" in response.text[:500].lower():
        response.encoding = "euc-jp"
    else:
        response.encoding = response.apparent_encoding or "utf-8"

    return response.text

def is_jra_race(race_id: str) -> bool:
    if len(race_id) >= 6:
        return race_id[4:6] in JRA_COURSE_CODES
    return False

print("✓ スクレイピング関数を定義しました")

In [ ]:
def scrape_race_list(target_date: date, jra_only: bool = True) -> List[Dict]:
    """指定日のレース一覧を取得（race_list_sub.htmlを使用）"""
    date_str = target_date.strftime("%Y%m%d")
    url = f"https://race.netkeiba.com/top/race_list_sub.html?kaisai_date={date_str}"

    html = fetch_html(url)
    soup = BeautifulSoup(html, "lxml")

    races = []
    seen_ids = set()

    for link in soup.find_all("a", href=True):
        href = link.get("href", "")
        match = re.search(r"race_id=(\d{12})", href)
        if match:
            race_id = match.group(1)
            if race_id not in seen_ids:
                if jra_only and not is_jra_race(race_id):
                    continue
                seen_ids.add(race_id)
                races.append({
                    "race_id": race_id,
                    "date": target_date.isoformat(),
                    "race_name": link.get_text(strip=True),
                })

    races.sort(key=lambda x: x["race_id"])
    return races


def get_odds_with_selenium(race_id: str, driver=None, max_retries: int = 2, expected_count: int = 0) -> Dict[int, Dict]:
    """Seleniumでnetkeibaオッズページからオッズを取得

    Args:
        race_id: レースID
        driver: Selenium WebDriver（Noneの場合は新規作成）
        max_retries: 最大リトライ回数（デフォルト2回）
        expected_count: 期待される頭数（0の場合はチェックしない）

    Returns:
        馬番をキーとしたオッズと人気の辞書
    """
    close_driver = False
    if driver is None:
        driver = get_chrome_driver()
        close_driver = True

    url = f"https://race.netkeiba.com/odds/index.html?race_id={race_id}&type=b1"
    best_result = {}

    for attempt in range(max_retries):
        try:
            driver.get(url)

            # オッズが読み込まれるまで待機
            try:
                WebDriverWait(driver, 15).until(
                    lambda d: len(d.find_elements(By.CSS_SELECTOR, "#odds_tan_block td.Odds span")) > 0
                )
            except:
                time.sleep(2.0)

            # 安定化待機
            time.sleep(1.5)

            result = {}
            
            # 標準セレクタで取得
            rows = driver.find_elements(By.CSS_SELECTOR, "#odds_tan_block table.RaceOdds_HorseList_Table tr")
            if not rows:
                rows = driver.find_elements(By.CSS_SELECTOR, "#odds_tan_block tr")

            for row in rows:
                try:
                    waku_elems = row.find_elements(By.CSS_SELECTOR, "td[class^='Waku']")
                    if not waku_elems:
                        continue
                    umaban_text = waku_elems[0].text.strip()
                    if not umaban_text.isdigit():
                        continue
                    umaban = int(umaban_text)

                    # オッズ取得
                    odds_text = None
                    odds_elem = row.find_elements(By.CSS_SELECTOR, "td.Odds span.Odds")
                    if odds_elem:
                        odds_text = odds_elem[0].text.strip()
                    if not odds_text or odds_text in ["---.-", "---", ""]:
                        odds_elem = row.find_elements(By.CSS_SELECTOR, "td.Odds span")
                        if odds_elem:
                            odds_text = odds_elem[0].text.strip()

                    if odds_text and odds_text not in ["---.-", "---", ""]:
                        try:
                            pop_elem = row.find_elements(By.CSS_SELECTOR, "td.Popular span")
                            popularity = None
                            if pop_elem:
                                pop_text = pop_elem[0].text.strip()
                                if pop_text.isdigit():
                                    popularity = int(pop_text)
                            result[umaban] = {
                                "odds": float(odds_text),
                                "popularity": popularity,
                            }
                        except ValueError:
                            pass
                except:
                    continue

            # 最良の結果を保持
            if len(result) > len(best_result):
                best_result = result.copy()

            # 成功判定：50%以上取得できたらOK（リトライしない）
            if result:
                if expected_count > 0:
                    coverage = len(result) / expected_count
                    if coverage >= 0.5:
                        return result
                    elif attempt < max_retries - 1:
                        # 50%未満の場合のみリトライ（ログ出力なし）
                        time.sleep(1.5)
                        continue
                else:
                    return result
            else:
                if attempt < max_retries - 1:
                    time.sleep(1.5)

        except Exception as e:
            if attempt == max_retries - 1:
                print(f"  ✗ オッズ取得失敗（{race_id}）: {e}")

    if close_driver:
        driver.quit()

    return best_result if best_result else {}


def scrape_shutuba(race_id: str, driver=None, odds_source: str = None) -> Dict:
    """出馬表を取得（レース前のデータ）"""
    url = f"https://race.netkeiba.com/race/shutuba.html?race_id={race_id}"

    response = session.get(url, timeout=30)
    if "EUC-JP" in response.text[:500] or "euc-jp" in response.text[:500].lower():
        response.encoding = "euc-jp"

    soup = BeautifulSoup(response.text, "lxml")

    race_data = {
        "race_id": race_id,
        "course": COURSE_NAMES.get(race_id[4:6], ""),
        "race_number": int(race_id[10:12]) if len(race_id) >= 12 else 0,
    }

    title_elem = soup.select_one(".RaceName")
    if title_elem:
        race_data["race_name"] = title_elem.get_text(strip=True)

    race_data_elem = soup.select_one(".RaceData01")
    if race_data_elem:
        text = race_data_elem.get_text()
        distance_match = re.search(r"(\d+)m", text)
        if distance_match:
            race_data["distance"] = int(distance_match.group(1))
        if "芝" in text:
            race_data["track_type"] = "芝"
        elif "ダート" in text or "ダ" in text:
            race_data["track_type"] = "ダート"

    entries = []
    for row in soup.select("tr.HorseList"):
        entry = parse_shutuba_row(row)
        if entry:
            entries.append(entry)

    field_size = len(entries)

    # オッズを取得
    odds_data = {}
    if USE_BROWSER and driver:
        odds_data = get_odds_with_selenium(race_id, driver, expected_count=field_size)

    if not odds_data:
        print(f"⚠ オッズ取得失敗: {race_id} - デフォルト値10.0を使用")

    # オッズをマージ
    for entry in entries:
        horse_num = entry.get("horse_number")
        if horse_num and horse_num in odds_data:
            entry["odds"] = odds_data[horse_num]["odds"]
            entry["popularity"] = odds_data[horse_num]["popularity"]

    race_data["entries"] = entries
    race_data["field_size"] = field_size
    return race_data


def parse_shutuba_row(row) -> Optional[Dict]:
    """出馬表の行をパース"""
    entry = {}

    tds = row.find_all("td")

    if len(tds) > 0:
        try:
            entry["frame_number"] = int(tds[0].get_text(strip=True))
        except:
            pass

    if len(tds) > 1:
        try:
            entry["horse_number"] = int(tds[1].get_text(strip=True))
        except:
            return None

    horse_link = row.select_one("span.HorseName a")
    if horse_link:
        href = horse_link.get("href", "")
        match = re.search(r"/horse/(\d+)", href)
        if match:
            entry["horse_id"] = match.group(1)
        entry["horse_name"] = horse_link.get_text(strip=True)

    jockey_td = row.select_one("td.Jockey a")
    if jockey_td:
        href = jockey_td.get("href", "")
        match = re.search(r"/jockey/(?:result/recent/)?(\d+)", href)
        if match:
            entry["jockey_id"] = match.group(1)
        entry["jockey_name"] = jockey_td.get_text(strip=True)

    if len(tds) > 5:
        try:
            entry["weight"] = float(tds[5].get_text(strip=True))
        except:
            pass

    return entry if entry.get("horse_number") else None


print("✓ 出馬表スクレイピング関数を定義しました")

In [ ]:
# 対象日を設定
target_date = date.today()  # 今日
# target_date = date(2025, 1, 12)  # 特定の日付

print(f"対象日: {target_date}")

# レース一覧を取得
races = scrape_race_list(target_date, jra_only=True)
print(f"\n{len(races)}件のJRAレースが見つかりました")

for race in races[:10]:
    print(f"  - {race['race_id']}: {race['race_name']}")

## 5. 特徴量の生成（事前計算済み特徴量を使用）

In [ ]:
def get_horse_features_batch(horse_ids: List[str]) -> Dict[str, Dict]:
    """複数馬の事前計算済み特徴量を一括取得"""
    if not horse_ids:
        return {}

    try:
        result = supabase.table("horse_features").select("*").in_("horse_id", horse_ids).execute()
        return {r["horse_id"]: r for r in result.data}
    except Exception as e:
        print(f"特徴量取得エラー: {e}")
        return {}

def get_jockey_features_batch(jockey_ids: List[str]) -> Dict[str, Dict]:
    """複数騎手の事前計算済み特徴量を一括取得"""
    if not jockey_ids:
        return {}

    try:
        result = supabase.table("jockey_features").select("*").in_("jockey_id", jockey_ids).execute()
        return {r["jockey_id"]: r for r in result.data}
    except Exception as e:
        print(f"騎手特徴量取得エラー: {e}")
        return {}

print("✓ 特徴量取得関数を定義しました")

In [ ]:
def create_features_v2(race_data: Dict, horse_features: Dict, jockey_features: Dict) -> pd.DataFrame:
    """
    レースデータと事前計算済み特徴量から予測用特徴量を生成

    事前計算済み特徴量 + レース固有特徴量 = 予測用特徴量
    """
    entries = race_data.get("entries", [])
    if not entries:
        return pd.DataFrame()

    features_list = []
    distance = race_data.get("distance", 1600)
    track_type = race_data.get("track_type", "芝")
    field_size = race_data.get("field_size", len(entries))

    # 季節特徴量
    race_date = date.today()
    month = race_date.month
    season = (month % 12) // 3 + 1  # 1:冬, 2:春, 3:夏, 4:秋

    for entry in entries:
        horse_id = entry.get("horse_id")
        jockey_id = entry.get("jockey_id")

        # 基本特徴量（レースから取得）
        features = {
            "horse_number": entry.get("horse_number"),
            "umaban": entry.get("horse_number"),
            "frame_number": entry.get("frame_number", 0),
            "weight": entry.get("weight", 55),
            "distance": distance,
            "track_type": 1 if track_type == "芝" else 0,
            "field_size": field_size,
            "race_number": race_data.get("race_number", 1),
            "odds": entry.get("odds", 10),
            "log_odds": np.log(entry.get("odds", 10) + 1),
            "popularity": entry.get("popularity", 10),
            # 季節
            "season": season,
            "month": month,
            "is_spring": 1 if 3 <= month <= 5 else 0,
            "is_summer": 1 if 6 <= month <= 8 else 0,
            "is_autumn": 1 if 9 <= month <= 11 else 0,
            "is_winter": 1 if month <= 2 or month == 12 else 0,
            "month_sin": np.sin(2 * np.pi * month / 12),
            "month_cos": np.cos(2 * np.pi * month / 12),
        }

        # 馬の事前計算済み特徴量を追加
        if horse_id and horse_id in horse_features:
            hf = horse_features[horse_id]
            for key in [
                "horse_age", "horse_sex",
                "avg_rank_last3", "avg_rank_last5", "avg_rank_last10", "avg_rank_all",
                "prize_3races", "prize_5races", "prize_10races",
                "win_rate", "place_rate", "show_rate", "best_rank", "total_runs",
                "days_since_last", "last_result",
                "avg_last3f", "best_last3f",
                "running_style", "avg_first_corner", "avg_last_corner",
                "position_up_avg", "escape_rate", "front_rate", "stalker_rate", "closer_rate",
                "avg_pace_first", "avg_pace_second", "avg_pace_diff", "pace_consistency",
                "high_pop_win_rate", "high_pop_show_rate", "high_pop_runs",
                "mid_pop_win_rate", "mid_pop_show_rate", "mid_pop_runs",
                "low_pop_win_rate", "low_pop_show_rate", "low_pop_runs",
                "avg_odds_when_win",
            ]:
                features[key] = hf.get(key)

            # 距離適性（レース距離に応じて選択）
            if distance <= 1400:
                features["distance_win_rate"] = hf.get("short_win_rate")
                features["distance_runs"] = hf.get("short_runs")
            elif distance <= 1800:
                features["distance_win_rate"] = hf.get("mile_win_rate")
                features["distance_runs"] = hf.get("mile_runs")
            elif distance <= 2200:
                features["distance_win_rate"] = hf.get("middle_win_rate")
                features["distance_runs"] = hf.get("middle_runs")
            else:
                features["distance_win_rate"] = hf.get("long_win_rate")
                features["distance_runs"] = hf.get("long_runs")

            # 芝/ダート適性
            if track_type == "芝":
                features["track_win_rate"] = hf.get("turf_win_rate")
                features["track_runs"] = hf.get("turf_runs")
            else:
                features["track_win_rate"] = hf.get("dirt_win_rate")
                features["track_runs"] = hf.get("dirt_runs")

        # 騎手の事前計算済み特徴量を追加
        if jockey_id and jockey_id in jockey_features:
            jf = jockey_features[jockey_id]
            features["jockey_win_rate"] = jf.get("win_rate")
            features["jockey_place_rate"] = jf.get("place_rate")
            features["jockey_show_rate"] = jf.get("show_rate")
            features["jockey_year_wins"] = jf.get("year_wins")
            features["jockey_year_rides"] = jf.get("year_rides")

        features_list.append(features)

    df = pd.DataFrame(features_list)

    # 足りない特徴量を0で埋める
    for col in feature_columns:
        if col not in df.columns:
            df[col] = 0

    return df

print("✓ 特徴量生成関数を定義しました")

## 6. 予測の実行

In [ ]:
def predict_race_v2(race_data: Dict, horse_features: Dict, jockey_features: Dict) -> Dict:
    """レースの予測を実行（v2: 事前計算済み特徴量使用）"""
    df = create_features_v2(race_data, horse_features, jockey_features)

    if df.empty:
        return None

    # 特徴量を選択
    X = df[feature_columns].copy()
    X = X.fillna(0)

    # スケーリング
    X_scaled = pd.DataFrame(
        scaler.transform(X),
        columns=X.columns,
        index=X.index,
    )

    # 予測（LightGBM二値分類は直接確率を出力）
    probs = model.predict(X_scaled)

    # 確率の正規化（レース内で合計が1になるように）
    # ※ソフトマックスは使わない - LightGBMは既に確率を出力しているため
    total = probs.sum()
    if total > 0:
        probabilities = probs / total
    else:
        probabilities = probs

    # 結果を整形
    results = []
    entries = race_data.get("entries", [])

    for i, entry in enumerate(entries):
        odds = entry.get("odds") or 10
        prob = float(probabilities[i])
        ev = prob * odds

        results.append({
            "horse_number": entry.get("horse_number"),
            "horse_name": entry.get("horse_name", "不明"),
            "jockey_name": entry.get("jockey_name", ""),
            "score": float(probs[i]),  # 元の確率を保持
            "probability": prob,
            "odds": odds,
            "expected_value": ev,
            "popularity": entry.get("popularity"),
        })

    # スコア順にソート
    results.sort(key=lambda x: x["score"], reverse=True)

    # 順位付け
    for i, r in enumerate(results):
        r["pred_rank"] = i + 1

    return {
        "race_id": race_data["race_id"],
        "race_name": race_data.get("race_name", ""),
        "course": race_data.get("course", ""),
        "race_number": race_data.get("race_number"),
        "distance": race_data.get("distance"),
        "track_type": race_data.get("track_type"),
        "predictions": results,
    }

print("✓ 予測関数を定義しました（バックエンドと同じロジック）")

## 7. 個別レース予測（JRA公式オッズ対応）

レースを指定して、JRA公式から最新オッズを取得して予測を実行します。

In [ ]:
# Playwright セットアップ（JRA公式からオッズ取得用）
print("Playwrightをセットアップ中...")

!pip install -q playwright nest_asyncio pytz
!playwright install chromium

import nest_asyncio
nest_asyncio.apply()  # Colab環境でasyncioを使えるようにする

import asyncio
import pytz
from playwright.async_api import async_playwright

# 日本時間（JST）
JST = pytz.timezone('Asia/Tokyo')

# JRA用の場所マッピング（IDから競馬場名への変換）
JRA_PLACE_MAPPING = {
    1: "札幌", 2: "函館", 3: "福島", 4: "新潟",
    5: "東京", 6: "中山", 7: "中京", 8: "京都",
    9: "阪神", 10: "小倉"
}


async def get_odds_from_jra_playwright(race_id: str, max_retries: int = 3) -> Dict:
    """
    PlaywrightでJRA公式サイトからリアルタイムオッズを取得（リトライ機能付き）

    Args:
        race_id: netkeibaのrace_id（例: 202608010801）
                 構造: [0:4]=年, [4:6]=場所, [6:8]=開催回, [8:10]=開催日, [10:12]=レース番号
        max_retries: 最大リトライ回数

    Returns:
        {
            "odds": {馬番: オッズ},
            "last_update": JRA側の最終更新時刻（文字列）,
            "fetch_time": 取得実行時刻（datetime, JST）
        }
    """
    # リンクテキストの生成
    place_code = int(race_id[4:6])
    kai = int(race_id[6:8])
    day = int(race_id[8:10])
    race_num = int(race_id[10:12])

    place_name = JRA_PLACE_MAPPING.get(place_code)
    if not place_name:
        print(f"⚠ JRA: 不明な競馬場コード: {place_code}")
        return {"odds": {}, "last_update": None, "fetch_time": datetime.now(JST)}

    kaisai_name = f"{kai}回{place_name}{day}日"
    race_name = f"{race_num}レース"

    print(f"  JRA公式からオッズ取得中: {kaisai_name} {race_name}...")

    for attempt in range(max_retries):
        fetch_time = datetime.now(JST)  # 日本時間で記録
        last_update = None
        
        try:
            async with async_playwright() as playwright:
                browser = await playwright.chromium.launch(headless=True)
                context = await browser.new_context()
                page = await context.new_page()
                
                # タイムアウト設定（モバイル環境を考慮）
                page.set_default_timeout(30000)

                try:
                    # JRAサイトにアクセス
                    await page.goto("https://www.jra.go.jp/keiba/", wait_until="domcontentloaded")
                    await asyncio.sleep(1.0)

                    # オッズページへ遷移
                    await page.get_by_role("link", name="オッズ", exact=True).click()
                    await asyncio.sleep(1.0)

                    # 開催選択
                    await page.get_by_role("link", name=kaisai_name).click()
                    await asyncio.sleep(1.0)

                    # レース選択（ページ遷移を待機）
                    async with page.expect_navigation():
                        await page.get_by_role("link", name=race_name).click()
                    await asyncio.sleep(1.0)

                    # HTMLを取得してパース
                    html = await page.content()
                    soup = BeautifulSoup(html, "lxml")
                    
                    # オッズ最終更新時刻を抽出（複数パターンに対応）
                    # パターン1: 「○時○分現在」
                    update_elem = soup.find(string=re.compile(r"\d+時\d+分現在"))
                    if update_elem:
                        last_update = update_elem.strip()
                    else:
                        # パターン2: テキスト内の時刻表記を探す
                        for text in soup.stripped_strings:
                            if "現在" in text:
                                match = re.search(r"(\d+時\d+分)", text)
                                if match:
                                    last_update = text.strip()
                                    break
                        # パターン3: 「○:○○現在」形式
                        if not last_update:
                            update_elem = soup.find(string=re.compile(r"\d+:\d+.*現在"))
                            if update_elem:
                                last_update = update_elem.strip()
                    
                    odds_df = pd.read_html(html)[0][["馬番", "単勝"]]

                    # 辞書形式に変換
                    result = {}
                    for _, row in odds_df.iterrows():
                        umaban = int(row["馬番"])
                        odds_value = row["単勝"]
                        if pd.notna(odds_value) and str(odds_value) not in ["---", "取消", "除外"]:
                            try:
                                result[umaban] = float(odds_value)
                            except:
                                pass

                    if result:
                        print(f"  ✓ JRA公式から{len(result)}頭のオッズを取得")
                        if last_update:
                            print(f"  ✓ オッズ最終更新: {last_update}")
                        
                        return {
                            "odds": result,
                            "last_update": last_update,
                            "fetch_time": fetch_time
                        }
                    else:
                        raise Exception("オッズが0件")

                finally:
                    await context.close()
                    await browser.close()

        except Exception as e:
            if attempt < max_retries - 1:
                print(f"  ⚠ JRA取得エラー、リトライ {attempt + 2}/{max_retries}: {e}")
                await asyncio.sleep(2.0)
            else:
                print(f"  ✗ JRA公式オッズ取得失敗: {e}")
    
    return {"odds": {}, "last_update": None, "fetch_time": datetime.now(JST)}


print("✓ Playwright準備完了（JRA公式オッズ取得用）")

In [ ]:
# レース一覧を表示
print("=== 本日のレース一覧 ===\n")
for i, race in enumerate(races):
    course = COURSE_NAMES.get(race["race_id"][4:6], "")
    race_num = int(race["race_id"][10:12])
    print(f"{i+1:2d}. {course}{race_num}R: {race['race_name']} (ID: {race['race_id']})")

In [ ]:
# 個別レース予測（JRA公式からリアルタイムオッズ取得）

# 推奨馬のフィルター条件
EV_THRESHOLD = 1.0   # 期待値下限
MAX_EV = 2.0         # 期待値上限（高すぎる穴馬を除外）
MIN_PROB = 0.05      # 最低確率5%
TOP_N_PER_RACE = 3   # レースごとの最大推奨数

# 上の一覧から番号を指定するか、race_idを直接入力
RACE_INDEX = 1  # @param {type:"integer"} 一覧の番号（1始まり）
# または直接race_idを指定（空欄の場合は上の番号を使用）
RACE_ID = ""  # @param {type:"string"}

# レースを特定
if RACE_ID:
    target_race_id = RACE_ID
else:
    if 1 <= RACE_INDEX <= len(races):
        target_race_id = races[RACE_INDEX - 1]["race_id"]
    else:
        print(f"エラー: 番号は1〜{len(races)}の範囲で指定してください")
        target_race_id = None

if target_race_id:
    print(f"予測対象: {target_race_id}")
    print(f"現在時刻: {datetime.now(JST).strftime('%H:%M:%S')} (JST)")
    print("-" * 50)
    
    # Seleniumドライバーを起動（フォールバック用）
    selenium_driver = None
    if USE_BROWSER:
        print("ChromeDriver起動中...")
        selenium_driver = get_chrome_driver()
        print("✓ ChromeDriver起動完了")
    
    try:
        # 1. netkeibaから基本データを取得（Seleniumでオッズも取得）
        print("\nnetkeibaから出馬表を取得中...")
        race_data = scrape_shutuba(target_race_id, driver=selenium_driver)
        
        # netkeibaオッズをバックアップとして保存
        netkeiba_odds = {}
        for entry in race_data.get("entries", []):
            horse_num = entry.get("horse_number")
            if horse_num and entry.get("odds"):
                netkeiba_odds[horse_num] = {
                    "odds": entry.get("odds"),
                    "popularity": entry.get("popularity")
                }
        
        # 2. JRA公式からPlaywrightでリアルタイムオッズを取得
        print("\nJRA公式から最新オッズを取得中...")
        jra_result = asyncio.get_event_loop().run_until_complete(
            get_odds_from_jra_playwright(target_race_id)
        )
        
        jra_odds = jra_result.get("odds", {})
        odds_last_update = jra_result.get("last_update")
        odds_fetch_time = jra_result.get("fetch_time")
        
        # 3. オッズをマージ（JRA優先、失敗時はnetkeiba）
        if jra_odds:
            odds_source_label = "JRA公式"
            for entry in race_data.get("entries", []):
                horse_num = entry.get("horse_number")
                if horse_num and horse_num in jra_odds:
                    entry["odds"] = jra_odds[horse_num]
            
            # オッズから人気を算出
            entries_with_odds = [(i, e.get("odds", 999)) for i, e in enumerate(race_data.get("entries", []))]
            entries_with_odds.sort(key=lambda x: x[1])
            for rank, (idx, _) in enumerate(entries_with_odds, 1):
                race_data["entries"][idx]["popularity"] = rank
            
            print(f"✓ JRA公式オッズをマージ完了")
        elif netkeiba_odds:
            odds_source_label = "netkeiba"
            print("⚠ JRA公式オッズ取得失敗 - netkeibaのオッズを使用")
        else:
            odds_source_label = "デフォルト"
            print("⚠ オッズ取得失敗 - デフォルト値10.0を使用")
        
        # 4. 特徴量取得
        horse_ids = [e.get("horse_id") for e in race_data.get("entries", []) if e.get("horse_id")]
        jockey_ids = [e.get("jockey_id") for e in race_data.get("entries", []) if e.get("jockey_id")]
        hf = get_horse_features_batch(horse_ids)
        jf = get_jockey_features_batch(jockey_ids)
        
        # 5. 予測実行
        pred = predict_race_v2(race_data, hf, jf)
        
        if pred:
            # 結果表示
            print(f"\n{'='*70}")
            print(f"{pred['course']} {pred.get('race_number', '')}R {pred['race_name']}")
            print(f"{pred.get('track_type', '')} {pred.get('distance', '')}m")
            print(f"オッズ取得元: {odds_source_label}")
            if jra_odds:
                if odds_last_update:
                    print(f"オッズ最終更新: {odds_last_update}")
                if odds_fetch_time:
                    print(f"オッズ取得時刻: {odds_fetch_time.strftime('%H:%M:%S')} (JST)")
            print(f"{'='*70}")
            print(f"{'予想':>4} {'馬番':>4} {'馬名':<14} {'勝率':>6} {'オッズ':>6} {'期待値':>6} {'人気':>4}")
            print("-" * 70)
            
            for p in pred["predictions"]:
                prob_pct = (p["probability"] or 0) * 100
                odds = p.get('odds')
                odds_str = f"{odds:.1f}" if odds else "-"
                ev = p.get("expected_value") or 0
                pop = p.get("popularity") or "-"
                
                # 推奨マーク（EV 1.0〜2.0、確率5%以上）
                if EV_THRESHOLD <= ev <= MAX_EV and (p.get("probability") or 0) >= MIN_PROB:
                    mark = "★"
                else:
                    mark = ""
                
                print(f"{p['pred_rank']:>4} {p['horse_number']:>4} {p['horse_name']:<14} {prob_pct:>5.1f}% {odds_str:>6} {ev:>5.2f}{mark:<2} {pop:>4}")
            
            # 推奨馬のみ抽出（シミュレーションと同じ条件）
            recommended = [p for p in pred["predictions"] 
                           if EV_THRESHOLD <= (p.get("expected_value") or 0) <= MAX_EV 
                           and (p.get("probability") or 0) >= MIN_PROB]
            if recommended:
                print(f"\n【推奨馬（EV {EV_THRESHOLD}〜{MAX_EV}、確率{MIN_PROB*100:.0f}%以上）】")
                for p in sorted(recommended, key=lambda x: x["expected_value"], reverse=True)[:TOP_N_PER_RACE]:
                    print(f"  馬番{p['horse_number']:2d} {p['horse_name']}: EV={p['expected_value']:.2f}, オッズ={p['odds']:.1f}")
        else:
            print("予測に失敗しました")
    
    finally:
        # Seleniumドライバーを終了
        if selenium_driver:
            selenium_driver.quit()
            print("\n✓ ChromeDriver終了")

---

## 使い方まとめ

### 事前準備（週1回、ローカルPCで実行）

1. **モデル学習**
   ```
   POST /api/v1/model/retrain
   ```

2. **モデルをSupabase Storageにアップロード**
   ```
   POST /api/v1/model/storage/upload
   ```

3. **特徴量をSupabase DBに同期**
   ```
   POST /api/v1/model/sync-features
   ```

### 当日予測（このノートブック）

1. セル1〜6を順番に実行（環境セットアップ〜予測関数定義）
2. セル7のレース一覧で対象レースの番号を確認
3. `RACE_INDEX` を変更して予測実行

### スマホからの使い方

1. Google Colabアプリをインストール
2. このノートブックを開く
3. セル1〜6を実行して準備
4. レース発走前にセル7で予測実行（最新オッズで計算）